In [231]:
from fuzzywuzzy import fuzz, process
fuzz.partial_ratio("-- And Now the Screaming Starts! [VHS]", "-- And Now the Screaming Starts!")

100

In [232]:
choices = ['"Black Lotus" Tibetan Dynamic Meditation', "America's Fighting Jets [DVD]"]
process.extract('"America\'s Dark Secrets" Documentary', choices)

[("America's Fighting Jets [DVD]", 54),
 ('"Black Lotus" Tibetan Dynamic Meditation', 41)]

In [233]:
import pandas as pd
movies_info = pd.read_csv('../Data/movies_info_full_date.csv')

In [237]:
movies_info['Title'].replace('\[VHS\]','',inplace=True,regex=True)
movies_info['Title'].replace('\[DVD\]','',inplace=True,regex=True)
movies_info['Title'].replace('VHS','',inplace=True,regex=True)
movies_info['Title'].replace('DVD','',inplace=True,regex=True)
movies_info['Title'].replace('\[Blu-ray\]','',inplace=True,regex=True)
movies_info['Title'].replace('\(with Movie Cash\)','',inplace=True,regex=True)
movies_info['Title'].replace('\(Home Use\)','',inplace=True,regex=True)
movies_info['Title'].replace('\(Rpkg\)','',inplace=True,regex=True)
movies_info['Title'].replace('\(Audition\)','',inplace=True,regex=True)
movies_info.replace(regex=r'^"(.*)"$', value=r'\1',inplace=True)
movies_info.sort_values(by=['Title'], ignore_index=True,inplace=True)

movies_info

,ASIN,Title,Language,Release date,Date First Available,Run time,Producers,Directors,Writers,Actors,Media Format,Subtitles,Genres


In [235]:
import datetime

i = 0

merged_movies_collection = pd.DataFrame(columns=['Title','Language','Release date','Date First Available','Run time','Producers','Directors','Writers','Actors','Media Format','Subtitles','Genres'])
merged_movies_source_asin_collection = pd.DataFrame(columns=['Title','Language','Release date','Date First Available','Run time','Producers','Directors','Writers','Actors','Media Format','Subtitles','Genres'])

whole_length = movies_info.shape[0]

for index, row in movies_info.iterrows():
    i += 1
    if whole_length - index <= 10: # 不能直接用shape[0]，因为会不断变化
        end = whole_length
    else:
        end = index + 10
        
    similar_title_info = process.extract(row['Title'], movies_info.loc[index:end, 'Title'] , limit = 10)
    similar_title_order = [s[2] for s in similar_title_info if s[1] >= 95]        
    
    # WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '-']
    # '$','-'
    if similar_title_order == [] and index in movies_info.index:
        similar_title_order = [index]
    # has been added
    else if similar_title_order == []:
        continue
    
    for_merge_info = movies_info.loc[similar_title_order,:]
    
    merged_movie_info = pd.DataFrame(data=[['',[],'','','','','',[],[],[],[],'']], columns=['Title','Language','Release date','Date First Available','Run time','Producers','Directors','Writers','Actors','Media Format','Subtitles','Genres'])
    merged_movie_source_asin = pd.DataFrame(data=[[[],[],[],[],'','','','',[],[],[],'']], columns=['Title','Language','Release date','Date First Available','Run time','Producers','Directors','Writers','Actors','Media Format','Subtitles','Genres'])
    
    # 初始
    for index, row in for_merge_info.iterrows():
        # Title 默认加上所有ASIN
        merged_movie_source_asin.loc[0,'Title'].append(row['ASIN'])
        # Title 选最短名
        if merged_movie_info.loc[0,'Title'] == '' or len(merged_movie_info.loc[0,'Title']) > len(row['Title']): 
            merged_movie_info.loc[0,'Title'] = row['Title']
        # Language 有且不重复就加进去
        if not pd.isna(row['Language']):
            merged_movie_source_asin.loc[0,'Language'].append(row['ASIN'])
            if row['Language'] not in merged_movie_info.loc[0,'Language']:
                merged_movie_info.loc[0,'Language'].append(row['Language'])
        # Release Date选最早的
        if not pd.isna(row['Release date']):
            if merged_movie_info.loc[0,'Release date'] == '':
                merged_movie_info.loc[0,'Release date'] = row['Release date']
                merged_movie_source_asin.loc[0,'Release date'].append(row['ASIN'])
            else:
                merged_date = datetime.datetime.strptime(merged_movie_info.loc[0,'Release date'],"%B %d, %Y")
                cur_date = datetime.datetime.strptime(row['Release date'],"%B %d, %Y")
                if merged_date > cur_date:
                    merged_movie_info.loc[0,'Release date'] = row['Release date']
                    merged_movie_source_asin.loc[0,'Release date'] = [row['ASIN']]
                elif merged_date == cur_date:
                    merged_movie_source_asin.loc[0,'Release date'].append(row['ASIN'])
        # Date First Available也是选最早的
        if not pd.isna(row['Date First Available']):
            if merged_movie_info.loc[0,'Date First Available'] == '':
                merged_movie_info.loc[0,'Date First Available'] = row['Date First Available']
                merged_movie_source_asin.loc[0,'Date First Available'].append(row['ASIN'])
            else:
                merged_date = datetime.datetime.strptime(merged_movie_info.loc[0,'Date First Available'],"%B %d, %Y")
                cur_date = datetime.datetime.strptime(row['Date First Available'],"%B %d, %Y")
                if merged_date > cur_date:
                    merged_movie_info.loc[0,'Date First Available'] = row['Date First Available']
                    merged_movie_source_asin.loc[0,'Date First Available'] = [row['ASIN']]
                elif merged_date == cur_date:
                    merged_movie_source_asin.loc[0,'Date First Available'].append(row['ASIN'])
        # Run time 选第一个出现的
        if not pd.isna(row['Run time']) and merged_movie_info.loc[0,'Run time'] == '':
            merged_movie_info.loc[0,'Run time'] = row['Run time']
            merged_movie_source_asin.loc[0,'Run time'] = row['ASIN']
        # Producers 同理
        if not pd.isna(row['Producers']) and merged_movie_info.loc[0,'Producers'] == '':
            merged_movie_info.loc[0,'Producers'] = row['Producers']
            merged_movie_source_asin.loc[0,'Producers'] = row['ASIN']
        # Directors 同理
        if not pd.isna(row['Directors']) and merged_movie_info.loc[0,'Directors'] == '':
            merged_movie_info.loc[0,'Directors'] = row['Directors']
            merged_movie_source_asin.loc[0,'Directors'] = row['ASIN']
        # Writers 同理 顺带加上split分组
        if not pd.isna(row['Writers']) and merged_movie_info.loc[0,'Writers'] == '':
            merged_movie_info.loc[0,'Writers'] = row['Writers'].split(',')
            merged_movie_source_asin.loc[0,'Writers'] = row['ASIN']
        # Actors 同理 顺带加上split分组
        if not pd.isna(row['Actors']) and merged_movie_info.loc[0,'Actors'] == []:
            merged_movie_info.loc[0,'Actors'] = row['Actors'].split(',')
            merged_movie_source_asin.loc[0,'Writer'] = row['ASIN']
        # Media Format 有且不重复就加进去
        if not pd.isna(row['Media Format']):
            merged_movie_source_asin.loc[0,'Media Format'].append(row['ASIN'])
            if row['Media Format'] not in merged_movie_info.loc[0,'Media Format']:
                merged_movie_info.loc[0,'Media Format'].append(row['Media Format'])
        # Subtitles 有且不重复就加进去
        if not pd.isna(row['Subtitles']):
            merged_movie_source_asin.loc[0,'Subtitles'].append(row['ASIN'])
            if row['Subtitles'] not in merged_movie_info.loc[0,'Subtitles']:
                merged_movie_info.loc[0,'Subtitles'].append(row['Subtitles'])
        # Genres 选第一个出现的
        if not pd.isna(row['Genres']) and merged_movie_info.loc[0,'Genres'] == '':
            merged_movie_info.loc[0,'Genres'] = row['Genres']
            merged_movie_source_asin.loc[0,'Genres'] = row['ASIN']
    
    merged_movies_collection =  pd.concat([merged_movies_collection,merged_movie_info], ignore_index=True)
    merged_movies_source_asin_collection = pd.concat([merged_movies_source_asin_collection, merged_movie_source_asin], ignore_index=True)
    
    movies_info.drop(similar_title_order, inplace=True)
    
    if i % 1000 == 0:
        print(i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [249]:
merged_movies_collection_without_empty = merged_movies_collection.loc[merged_movies_collection['Title'].map(lambda d: len(d)) > 0]

In [247]:
merged_movies_source_asin_collection_without_empty = merged_movies_source_asin_collection.loc[merged_movies_source_asin_collection['Title'].map(lambda d: len(d)) > 0]

In [250]:
merged_movies_collection_without_empty

,Title,Language,Release date,Date First Available,Run time,Producers,Directors,Writers,Actors,Media Format,Subtitles,Genres
0,- Learn To Sing Gospel,[],"February 1, 2007",,2 hours,,"Greg Hannon, M.M.",[],"[Canagela Robertson, M.M.]",[NTSC],[],
1,- Learning To Flatpick,[],"June 3, 2003",,4 hours,,"Traum, Happy",[],"[Kaufman, Steve]","[Multiple Formats, Color, NTSC, Closed-captioned]",[],
2,100 Boating Mistakes & How To Avoid Them Trai...,[],"January 1, 2007",,50 minutes,,,[],[],"[Multiple Formats, Color, NTSC]",[],
3,100 Sailing Mistakes & How To Avoid Them Trai...,[],"January 1, 2007",,50 minutes,James Marsh,,[],[],"[Color, DVD, NTSC]",[],
4,100 Saltwater Fishing Mistakes & How To Avoid...,[],"January 1, 2007",,50 minutes,James Marsh,,[],[],"[Multiple Formats, Color, NTSC]",[],
...,...,...,...,...,...,...,...,...,...,...,...,...
233123,Águila o sol,[Spanish],"June 30, 1996","September 29, 2006",,,Arcady Boytler,[],"[Cantinflas, Manuel Medel, Margarita Mora, ...",[],[],
233124,Æon Flux,[],"June 3, 2008",,1 hour and 33 minutes,,Karyn Kusama,[],"[Charlize Theron, Frances McDormand, Sophie ...","[Multiple Formats, Blu-ray, Color, Dolby, NTSC...","[English, French, Spanish]",
233125,ÇA TWISTE À POPONGUINE (Rocking Poponguine),[],"September 10, 2009",,,,Moussa Sene Absa,[],"[Coura Ba, Jean-François Balmer, Ousmane Bo]",[Available to watch on ],[],Arthouse
233127,Ça commence aujourd'hui,"[French (Dolby Digital 5.1), Italian (Dolby Di...","January 21, 2003","June 27, 2006",1 hour and 57 minutes,"Alain Sarde, Frédéric Bourboulon",Bertrand Tavernier,[],"[Philippe Torreton, Maria Pitarresi, Nadia K...",[PAL],"[German, Dutch, English]",


In [251]:
merged_movies_source_asin_collection_without_empty

,Title,Language,Release date,Date First Available,Run time,Producers,Directors,Writers,Actors,Media Format,Subtitles,Genres,Writer
0,[B000O1OWQS],[],[B000O1OWQS],[],B000O1OWQS,,B000O1OWQS,,[],[B000O1OWQS],[],,B000O1OWQS
1,[B000092OM6],[],[B000092OM6],[],B000092OM6,,B000092OM6,,[],[B000092OM6],[],,B000092OM6
2,[B0004ILML4],[],[B0004ILML4],[],B0004ILML4,,,,[],[B0004ILML4],[],,NaN
3,[B0004IXFE6],[],[B0004IXFE6],[],B0004IXFE6,B0004IXFE6,,,[],[B0004IXFE6],[],,NaN
4,[B0004ILMKA],[],[B0004ILMKA],[],B0004ILMKA,B0004ILMKA,,,[],[B0004ILMKA],[],,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233123,[6302296285],[6302296285],[6302296285],[6302296285],,,6302296285,,[],[],[],,6302296285
233124,[B000IOM0WE],[],[B000IOM0WE],[],B000IOM0WE,,B000IOM0WE,,[],[B000IOM0WE],[B000IOM0WE],,B000IOM0WE
233125,"[B002SUHWDI, B002SUDU48]",[],"[B002SUHWDI, B002SUDU48]",[],,,B002SUHWDI,,[],"[B002SUHWDI, B002SUDU48]",[],B002SUHWDI,B002SUHWDI
233127,"[B00004VYF2, B00004D2WE]","[B00004VYF2, B00004D2WE]","[B00004VYF2, B00004D2WE]",[B00004D2WE],B00004VYF2,B00004VYF2,B00004VYF2,,[],[B00004VYF2],[B00004VYF2],,B00004VYF2


In [252]:
merged_movies_collection_without_empty.to_csv('../Data/merged_title_movies_collection.csv', index = False, sep = ',')
merged_movies_source_asin_collection_without_empty.to_csv('../Data/merged_title_movies_source_asin_collection.csv', index = False, sep = ',')